In [1]:
import rasterio
import pandas as pd
import numpy as np
import pyproj
import os

In [3]:
points = pd.read_csv('../../lora/data/shp/patch_coords.csv')

source_crs = pyproj.CRS("EPSG:4326")  # WGS 84
target_crs = pyproj.CRS("EPSG:25833")  # UTM zone 33N

transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

points["x"], points["y"] = zip(*points.apply(lambda row: transformer.transform(row["lon"], row["lat"]), axis=1))

In [5]:
#only actual point value used
planet_points = None

in_path = '../rasters/planet_tifs/'

for folder in os.listdir(in_path):
    for filename in os.listdir(os.path.join(in_path, folder)):
        pixel_values = []
        with rasterio.open(os.path.join(in_path, folder, filename)) as src:
            # Extract pixel values
            for _, point in points.iterrows():
                if src.count == 4: #for 4 band images
                    for band in range(1, src.count+1):
                        row, col = src.index(point['x'], point['y'])
                        value = src.read(band)[row, col]
                        bandn = 2*band
                        pixel_values.append([point['patch'], folder, bandn, value])
                else: #for 8 band images
                    for band in range(1, src.count+1):
                        row, col = src.index(point['x'], point['y'])
                        value = src.read(band)[row, col]
                        pixel_values.append([point['patch'], folder, band, value])

            # Organize data
            columns = ['patch', 'date', 'band', 'pixel_value']
            pixel_df = pd.DataFrame(pixel_values, columns=columns)
            if planet_points is None:
                planet_points = pixel_df
            else:
                planet_points = pd.concat([planet_points, pixel_df])

#Pivot columns to wider
planet_points_single = planet_points.pivot_table(index=['patch','date'],
                                                 columns='band',
                                                 values='pixel_value')
planet_points_single.rename(columns={col: f'band_{col}' for col in planet_points_single.columns if str(col).isdigit()}, inplace=True)
planet_points_single.reset_index(inplace=True)
planet_points_single['date'] = pd.to_datetime(planet_points_single['date'], format='%Y-%m-%d')

# output as csv
planet_points_single.to_csv("merge_ready/planet_mr.csv", index = False)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../rasters/planet_train_tifs/'